※コンテストURL：https://atcoder.jp/contests/joi2011ho

In [1]:
from ipywidgets import Textarea
import io

if 'open' in globals():
    del open
if 'input' in globals():
    del input

original_open = open

class custom_open():
    def __init__(self):
        self.text = ''

    def __call__(self, file, *args, **kwargs):
        if file == 0:
            return io.StringIO(self.text)
        return original_open(file, *args, **kwargs)

    def updater(self, change):
        self.text = change["new"]

class custom_input():
    def __init__(self):
        self.__sio = io.StringIO('')
        self.shell = get_ipython()
        if self.shell.events.callbacks['pre_run_cell'] != []:
            self.shell.events.callbacks['pre_run_cell'] = []
        self.shell.events.register('pre_run_cell', self.pre_run_cell)

    def __call__(self):
        return self.__sio.readline().strip()

    def pre_run_cell(self, info):
        text = self.shell.user_ns.get('text_area', None).value
        self.__sio = io.StringIO(text)

open = custom_open()
input = custom_input()

text_area = Textarea()
text_area.observe(open.updater, names='value')
display(text_area)

Textarea(value='')

# A問題

In [9]:
from itertools import accumulate
import numpy as np

M, N = map(int, input().split())
K = int(input())
field = [list(input()) for _ in range(M)]

S = [[np.zeros(3, dtype=int) for _ in range(N+1)] for _ in range(M+1)]
dic = {"J":0, "O":1, "I":2}

for i in range(M):
    for j in range(N):
        t = dic[field[i][j]]
        S[i+1][j+1] = S[i][j+1] + S[i+1][j] - S[i][j]
        S[i+1][j+1][t] += 1
                
for _ in range(K):
    a,b,c,d = map(int, input().split())
    ans = S[c][d] - S[c][b-1] - S[a-1][d] + S[a-1][b-1]
    print(*ans)

1 3 2
3 5 2
0 1 0
10 11 7


上のコードだとpythonだと間に合わない（C++だと同様の実装でいける）らしい？　供養。  
以下、pythonでも通るコード。

In [27]:
import sys
import numpy as np
read = sys.stdin.buffer.read
readline = sys.stdin.buffer.readline
readlines = sys.stdin.buffer.readlines

H, W = map(int, readline().split())
K = int(readline())
grid = np.zeros((H+1, W+1), 'S1')
for i in range(1, H+1):
    grid[i, 1:] = np.frombuffer(readline().rstrip(), 'S1')
Q = np.array(read().split(), np.int32)

J = (grid == b'J').cumsum(axis=0).cumsum(axis=1)
O = (grid == b'O').cumsum(axis=0).cumsum(axis=1)

del grid

A = Q[::4] - 1
B = Q[1::4] - 1
C = Q[2::4]
D = Q[3::4]

J = J[C, D] + J[A, B] - J[C, B] - J[A, D]
O = O[C, D] + O[A, B] - O[C, B] - O[A, D]
I = (C-A) * (D-B) - J - O

del Q, A, B, C, D

print('\n'.join('{} {} {}'.format(j, o, i) for j, o, i in zip(J, O, I)))
